In [1]:
import numpy as np
import cv2
import time


"""
Start of:
Reading stream video from camera
"""

camera = cv2.VideoCapture(0)

h, w = None, None

"""
End of:
Reading stream video from camera
"""


"""
Start of:
Loading YOLO v3 network
"""


with open('yolo-coco-data/coco.names') as f:

    labels = [line.strip() for line in f]



network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yolov3.cfg',
                                     'yolo-coco-data/yolov3.weights')

layers_names_all = network.getLayerNames()

layers_names_output = \
    [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

probability_minimum = 0.5


threshold = 0.3

colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')



"""
End of:
Loading YOLO v3 network
"""


"""
Start of:
Reading frames in the loop
"""

while True:
    _, frame = camera.read()

    if w is None or h is None:

        h, w = frame.shape[:2]

    """
    Start of:
    Getting blob from current frame
    """


    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)

    """
    End of:
    Getting blob from current frame
    """

    """
    Start of:
    Implementing Forward pass
    """

    network.setInput(blob)  # setting blob as input to the network
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()


    print('Current frame took {:.5f} seconds'.format(end - start))

    """
    End of:
    Implementing Forward pass
    """

    """
    Start of:
    Getting bounding boxes
    """

    bounding_boxes = []
    confidences = []
    class_numbers = []

    for result in output_from_network:
        for detected_objects in result:
            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]


            if confidence_current > probability_minimum:

                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                bounding_boxes.append([x_min, y_min,
                                       int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)

    """
    End of:
    Getting bounding boxes
    """

    """
    Start of:
    Non-maximum suppression
    """

    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                               probability_minimum, threshold)

    """
    End of:
    Non-maximum suppression
    """

    """
    Start of:
    Drawing bounding boxes and labels
    """

    if len(results) > 0:
        for i in results.flatten():

            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            colour_box_current = colours[class_numbers[i]].tolist()

            cv2.rectangle(frame, (x_min, y_min),
                          (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)

            text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                                   confidences[i])

            cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)

    """
    End of:
    Drawing bounding boxes and labels
    """

    """
    Start of:
    Showing processed frames in OpenCV Window
    """


    cv2.namedWindow('YOLO v3 Real Time Detections', cv2.WINDOW_NORMAL)
    cv2.imshow('YOLO v3 Real Time Detections', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    """
    End of:
    Showing processed frames in OpenCV Window
    """

"""
End of:
Reading frames in the loop
"""


camera.release()
cv2.destroyAllWindows()

Current frame took 2.10887 seconds
Current frame took 0.68916 seconds
Current frame took 0.69614 seconds
Current frame took 0.60279 seconds
Current frame took 0.55280 seconds
Current frame took 0.56559 seconds
Current frame took 0.49818 seconds
Current frame took 0.56932 seconds
Current frame took 0.52958 seconds
Current frame took 0.52460 seconds
Current frame took 0.51606 seconds
Current frame took 0.51798 seconds
Current frame took 0.54805 seconds
Current frame took 0.52898 seconds
Current frame took 0.61257 seconds
Current frame took 0.54595 seconds
Current frame took 0.54146 seconds
Current frame took 0.56962 seconds
Current frame took 0.67530 seconds
Current frame took 0.59974 seconds
Current frame took 0.54395 seconds
Current frame took 0.60049 seconds
Current frame took 0.60837 seconds
Current frame took 0.53913 seconds
Current frame took 0.51448 seconds
Current frame took 0.65401 seconds
Current frame took 0.61146 seconds
Current frame took 0.55541 seconds
Current frame took 0

"\nSome comments\n\ncv2.VideoCapture(0)\n\nTo capture video, it is needed to create VideoCapture object.\nIts argument can be camera's index or name of video file.\nCamera index is usually 0 for built-in one.\nTry to select other cameras by passing 1, 2, 3, etc.\n"